# Загрузка данных

In [2]:
import numpy as np 
import pandas as pd 

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split

import tqdm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
downloaded = drive.CreateFile({'id':"1AFJdxwFCSOyCVFmtmlaUzY9OunwbAVVZ"}) 
downloaded.GetContentFile('tags.csv')
tag=pd.read_csv('tags.csv')

In [5]:
downloaded = drive.CreateFile({'id':"1PZZUhKWPmRCzgwJP9yX5g7ZycV273oVk"}) 
downloaded.GetContentFile('rating.csv')
rating=pd.read_csv('rating.csv')

In [6]:
downloaded = drive.CreateFile({'id':"1Wgw78UNg1uPV238DH0Oj3QYLfgiU7qpg"}) 
downloaded.GetContentFile('movies.csv')
movies=pd.read_csv('movies.csv')

In [7]:
downloaded = drive.CreateFile({'id':"1k4ZwrBeCO80LHThymYzHfD_H-gGCaMr8"}) 
downloaded.GetContentFile('genome-scores.csv')
genome_scores=pd.read_csv('genome-scores.csv')

In [8]:
downloaded = drive.CreateFile({'id':"12lE3TfDja2VPrqbHpXc92m2r0Sdf0Sog"}) 
downloaded.GetContentFile('genome-tags.csv')
genome_tag=pd.read_csv('genome-tags.csv')

In [9]:
downloaded = drive.CreateFile({'id':"1kx_nH4Sv9h1h9fXoLN6531YZWsDxXqO-"}) 
downloaded.GetContentFile('links.csv')
link=pd.read_csv('links.csv')
del(downloaded)

# Задание 2.1

Будем делать рекомендации не всем и сразу, а конкретному пользователю. Формировать movie_pivot будем только для этого конкретного пользователя.

1. Возьмём все оценки, которые поставил пользователь  U .
2. По фильмам  MU , которые он оценил, получим всех пользователей, которые ставили этим фильмам оценки.
3. Скорее всего и на этом этапе список получится большим (для случайного пользователя у меня получилось 1.7 миллионов строк). Теперь, например, можем оставить только пользователей, у которых много общих фильмов.
4. Для каждого пользователя посчитаем похожесть (например, корреляцию).
5. Отсортируем пользователей по похожести.
6. Оставим только  K  самых похожих пользователей.

нужно оценить качество этой рекомендации. Идея тут такая: разделить исходные данные на train и test. Получить предсказания на train и проверить их на test.

## Поиск похожих пользователей

In [44]:
k=8
movies_leave_percent = 0.3
test_data_percent = 0.3

In [45]:
all_users = rating['userId'].value_counts()
user = all_users.sample(1).index[0]
user

63998

In [46]:
def get_seen(userId):
  return df.movieId[df.userId==userId].values

def get_unseen(userId):
  s = df.movieId[df.userId==userId].values
  return movies.movieId[movies.movieId.isin(s)==False].values

In [47]:
df = pd.merge(movies,rating)
df.drop("timestamp", 1, inplace=True)
print(df.shape)
df[-2:]

(25000095, 5)


,movieId,title,genres,userId,rating
25000093,209169,A Girl Thing (2001),(no genres listed),119571,3.0
25000094,209171,Women of Devil's Island (1962),Action|Adventure|Drama,119571,3.0


In [48]:
user_movies_l = get_seen(user)
user_movies = df[df.userId==user][["movieId","title","rating"]]

In [61]:
train, test = train_test_split(user_movies, test_size=test_data_percent)

In [62]:
users_movies = df[df.movieId.isin(train.movieId)][["movieId","title","userId","rating"]]

In [63]:
users_group = users_movies.groupby(["userId"])

In [64]:
about_users_grp = users_movies.userId.value_counts()
about_users_grp

20055     32
63998     32
80974     32
92046     32
8619      31
          ..
113899     1
96895      1
9630       1
11679      1
2047       1
Name: userId, Length: 144228, dtype: int64

Ограничиваем количество пользователей, для которых считаем близость, выбирая количество 70% от всех пользователей.

In [65]:
about_users = pd.DataFrame(about_users_grp)
s = about_users[about_users.userId>=round(len(user_movies_l)*movies_leave_percent)].shape[0]
about_users = about_users[:s]
about_users.columns = ["count"]
about_users

,count
20055,32
63998,32
80974,32
92046,32
8619,31
...,...
89870,14
17284,14
59896,14
109372,14


In [72]:
res = []
for i in tqdm(range(s)):
  next_user = about_users_grp.index[i]
  # выберем список одинаковых групп двух пользователей
  temp = train[train['movieId'].isin(users_group.get_group(next_user)['movieId'])].sort_values(by='movieId')
  person1 = temp.sort_values(by='movieId')
  person2 = users_group.get_group(next_user).sort_values(by='movieId')
  res.append(pearsonr(person1.rating, person2.rating)[0])

 30%|██▉       | 2770/9278 [00:07<00:16, 397.77it/s]/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
100%|██████████| 9278/9278 [00:23<00:00, 386.77it/s]


In [73]:
about_users["similarity"] = res
susers = about_users.sort_values(by="similarity", ascending=False)[1:k+1].reset_index()
susers.columns = ["userId", "rating_count", "similarity"]
susers

,userId,rating_count,similarity
0,107513,14,0.909351
1,138854,14,0.818433
2,61714,17,0.802434
3,85675,19,0.788461
4,98364,15,0.785128
5,79711,14,0.780675
6,56969,15,0.777683
7,75251,15,0.772511


In [74]:
susers = df.merge(susers, left_on='userId', right_on='userId', how='inner')
susers['weightedRating'] = susers['similarity']*susers['rating']
print(susers.shape)
susers[:2]

(5601, 8)


,movieId,title,genres,userId,rating,rating_count,similarity,weightedRating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,56969,4.0,15,0.777683,3.110731
1,2,Jumanji (1995),Adventure|Children|Fantasy,56969,3.0,15,0.777683,2.333048


Оценить точность прогноза мы можем только на тех товарах, которые пользователь уже оценил. 

Стандартные подходы - кросс-валидация методами:
*   leave-one-out — модель обучается на всех оцененных пользователем объектах, кроме одного, а тестируется на этом одном объекте. Так делается для всех n объектов, и среди полученных n оценок качества вычисляется среднее.
*   leave-p-out — то же самое, но на каждом шаге исключается p точек.

Все метрики качества можно условно разделить на три категории:
*   Prediction Accuracy — оценивают точность предсказываемого рейтинга по непрерывной шкале (0-10) (MAE, MSE, RMSE),
*   Decision support — оценивают релевантность рекомендаций по бинарным данным (Precision, Recall, F1-Measure, ROC AUC, и др),
*   Rank Accuracy метрики — оценивают качество ранжирования выдаваемых рекомендаций - насколько правилен порядок показа рекомендаций в отсортированном списке.

В нашем случае baseline алгоритмом, с которым будем сравнивать превосходство в точности, может быть просто отсутствие рекомендаций. 



In [69]:
'''
susers_train, susers_test = train_test_split(susers, test_size=0.3)
susers_test.drop(columns=["rating_count", "similarity", "weightedRating"], inplace=True)
print('Train shape: {}'.format(susers_train.shape))
print('Test shape: {}'.format(susers_test.shape))
'''

Train shape: (3920, 8)
Test shape: (1681, 5)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [71]:
train[:2]

,movieId,title,rating
5051908,904,Rear Window (1954),5.0
17936492,6944,Father of the Bride (1991),4.0


In [75]:
temp_susers = susers.groupby('movieId').sum()[['similarity','weightedRating']]
print(temp_susers.shape)
temp_susers[:2]

(2783, 2)


,similarity,weightedRating
movieId,,
1,6.434674,25.699546
2,4.813807,11.560920


In [76]:
recommendation_df = pd.DataFrame()
recommendation_df['score'] = temp_susers['weightedRating']/temp_susers['similarity']
recommendation_df['movieId'] = temp_susers.index
recommendation_df = recommendation_df.sort_values(by='score', ascending=False)
recommendation_df.reset_index(drop=True, inplace=True)
recommendation_df

,score,movieId
0,5.0,3629
1,5.0,2905
2,5.0,3030
3,5.0,2208
4,5.0,1281
...,...,...
2778,0.5,1556
2779,0.5,81132
2780,0.5,56176
2781,0.5,54004


In [77]:
check_df = pd.merge(susers_test, recommendation_df, on='movieId')
check_df

,movieId,title,genres,userId,rating,score
0,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,98364,4.5,4.003952
1,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,138854,4.0,4.003952
2,32587,Sin City (2005),Action|Crime|Film-Noir|Mystery|Thriller,98364,4.0,3.747009
3,44,Mortal Kombat (1995),Action|Adventure|Fantasy,85675,2.5,2.375136
4,44,Mortal Kombat (1995),Action|Adventure|Fantasy,98364,2.0,2.375136
...,...,...,...,...,...,...
1676,170,Hackers (1995),Action|Adventure|Crime|Thriller,107513,4.5,3.710322
1677,78637,Shrek Forever After (a.k.a. Shrek: The Final C...,Adventure|Animation|Children|Comedy|Fantasy|IMAX,107513,2.5,2.500000
1678,3481,High Fidelity (2000),Comedy|Drama|Romance,98364,3.5,3.496509
1679,1952,Midnight Cowboy (1969),Drama,85675,3.0,3.506298


In [78]:
mse = sqrt(mean_squared_error(check_df.rating, check_df.score))
print(mse, "- RMSE (Root Mean Squared Error)")

0.6606814313576325 - RMSE (Root Mean Squared Error)


Предсказание оказалось точное, что получилось засчет нахождения действительно близких по вкусам пользователей.